# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv("../output_data/cities.csv")

city_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key = g_key)

#Creating variables for Location and Humidity
locations = city_weather[["Lat", "Lng"]]
Humidity = city_weather["Humidity"]

#Define the heatmap parameters
fig_map = gmaps.figure()

#Create heat layer
heatmap_layer=gmaps.heatmap_layer(locations, weights=Humidity, dissipating= False, max_intensity=250,
                              point_radius=3)

#Add Layer
fig_map.add_layer(heatmap_layer)

#Display the map
fig_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
city_df = city_weather.loc[(city_weather["Wind Speed"] <= 20) & 
                           (city_weather["Cloudiness"] == 20) & \
                           (city_weather["Max Temp"] >= 70) & 
                           (city_weather["Max Temp"] <= 90)].dropna()

city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.8,6.93
54,54,tiarei,20,PF,1558378764,70,-17.53,-149.33,84.2,10.29
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.6,14.99
89,89,ternate,20,PH,1558378770,74,14.29,120.72,86.0,1.12
103,103,kapaa,20,US,1558378773,74,22.08,-159.32,80.6,10.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create a hotel_df
hotel_df = city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
25,kantang,TH,7.42,99.52,
54,tiarei,PF,-17.53,-149.33,
68,beja,PT,38.02,-7.87,
89,ternate,PH,14.29,120.72,
103,kapaa,US,22.08,-159.32,
153,porto novo,BR,-23.68,-45.44,
188,lagoa,PT,37.14,-8.45,
201,luanda,AO,-8.83,13.24,
218,salalah,OM,17.01,54.10,
223,koronowo,PL,53.31,17.94,


In [6]:
#set url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#search params
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [7]:
for index, row in hotel_df.iterrows():
    #get city name, latitude, longitude from hotl_df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    #add params
    params["location"] = f"{lat},{lng}"

    #API requests
    print(f"Retrieving Data {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #Save data
    results = response['results']
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    #no hotel errors
    except (KeyError, IndexError):
         print("Missing data. skipping...")    
    
    print("_____________________________________________________________________")

#search completed    
print("Data Retrieval Completed")

Retrieving Data 25: kantang.
Missing data. skipping...
_____________________________________________________________________
Retrieving Data 54: tiarei.
Closest hotel in tiarei is Le Rocher de Tahiti.
_____________________________________________________________________
Retrieving Data 68: beja.
Closest hotel in beja is BejaParque Hotel.
_____________________________________________________________________
Retrieving Data 89: ternate.
Missing data. skipping...
_____________________________________________________________________
Retrieving Data 103: kapaa.
Closest hotel in kapaa is Hilton Garden Inn Kauai Wailua Bay.
_____________________________________________________________________
Retrieving Data 153: porto novo.
Closest hotel in porto novo is VaiDaPesca Hotel & Marina.
_____________________________________________________________________
Retrieving Data 188: lagoa.
Closest hotel in lagoa is Tivoli Carvoeiro Algarve Resort.
_________________________________________________________

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
25,kantang,TH,7.42,99.52,
54,tiarei,PF,-17.53,-149.33,Le Rocher de Tahiti
68,beja,PT,38.02,-7.87,BejaParque Hotel
89,ternate,PH,14.29,120.72,
103,kapaa,US,22.08,-159.32,Hilton Garden Inn Kauai Wailua Bay
153,porto novo,BR,-23.68,-45.44,VaiDaPesca Hotel & Marina
188,lagoa,PT,37.14,-8.45,Tivoli Carvoeiro Algarve Resort
201,luanda,AO,-8.83,13.24,EPIC SANA Luanda
218,salalah,OM,17.01,54.10,Al Baleed Resort Salalah by Anantara
223,koronowo,PL,53.31,17.94,Ośrodek Wypoczynkowy Nad Zatoką


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display figure
fig_map.add_layer(markers)
fig_map

Figure(layout=FigureLayout(height='420px'))